# Facebook AI Similarity Search
Vector Store do Facebook

In [8]:
from langchain_community.document_loaders.pdf import PyPDFLoader

path = '../documents/guia_alimentar_populacao_brasileira.pdf'
loader = PyPDFLoader(path)

pdf_pages = loader.load()

### Faz splitting do PDF:

In [9]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(
    chunk_size=700,
    chunk_overlap=70,
    separators=["\n\n", "\n", ".", " ", ""]
)

documents = splitter.split_documents(pdf_pages)

### Criando a Vector Store persistida em arquivo

In [10]:
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores.faiss import FAISS

embedding_model = OpenAIEmbeddings(model="text-embedding-ada-002")

chroma_location = "../documents/chroma_vectorstore/"

vector_store = FAISS.from_documents(
    documents=documents,
    embedding=embedding_model
)

In [12]:
question = "O que fala sobre macaxeira?"

docs = vector_store.similarity_search(question, k=5)

for doc in docs:
    print(doc.page_content)
    print(f'======== {doc.metadata} \n')

71 70     GUIA ALIMENTAR PARA A POPULAÇÃO BRASILEIRA
GrUPo das raízes e tUBérCULos
este grupo inclui a mandioca, também 
conhecida como macaxeira ou aipim, batata ou batata-inglesa, batata-doce, batata-baroa ou mandioquinha, cará e inhame.
raízes e tubérculos são alimentos muito 
versáteis, podendo ser feitos cozidos, assados, ensopados ou na forma de purês. são frequentemente consumidos pelos brasileiros no almoço e no jantar, junto com feijão e arroz, legumes e carnes. em algumas regiões do Brasil, a mandioca e a batata-doce são consumidas no café da manhã como substitutos do pão. a mandioca, em particular, também é usada no preparo de doces caseiros como pudins e bolos.
======== {'source': '../documents/guia_alimentar_populacao_brasileira.pdf', 'page': 71} 

Sopa de legumes, farinha de mandioca e açaíArroz, feijão, fígado bovino e abobrinha refogadaSalada de folhas, arroz, feijão, ovo e maçã
Salada de folhas, macarrão e galeto
======== {'source': '../documents/guia_alimentar_populac

### Salvando local FAISS

In [13]:
vector_store.save_local("../documents/faiss_vectorstore/")

### Carregando de arquivo

In [16]:
vector_store = FAISS.load_local(
    "../documents/faiss_vectorstore/", 
    embeddings=embedding_model, 
    allow_dangerous_deserialization=True
)